In [98]:
%load_ext autoreload
%autoreload 1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [99]:
%%file GRBL.py
import serial
class GRBL(object):
    BAUDRATE = 115200
    
    def __init__(self, port):
        self.serial = serial.Serial(port=port,
                                    baudrate=GRBL.BAUDRATE,
                                    timeout=0.10)
        
    def write(self, command_line=""):
        self.serial.flushInput()
        self.serial.write("\n".encode())
        self.serial.write("{cmd}\n".format(cmd=command_line).encode())
        
    def read(self, multiline=True):
        if multiline:
            responses = self.serial.readlines()
            responses = [response.decode().strip() for response in responses]
            return responses
        else:
            response = self.serial.readline()
            return response.decode().strip()
 
    def cmd(self, command_line, resp=True, multiline=True):
        self.write(command_line)
        if resp:
            return self.read(multiline=multiline)
        return None
    
    def reset(self):
        """ https://github.com/gnea/grbl/wiki/Grbl-v1.1-Commands#grbl-v11-realtime-commands
        """
        ret = self.cmd("\x18")
        assert(ret[-1]=='ok')
        
    def sleep(self):
        """ https://github.com/gnea/grbl/wiki/Grbl-v1.1-Commands#slp---enable-sleep-mode
        """
        ret = self.cmd("$SLP")
        assert(ret[-1]=='ok')
    
    @property
    def status(self):
        """
        """
        ret = self.cmd("?")
        assert(ret[-1]=='ok')
        return ret[1]
        
    def kill_alarm(self):
        """ https://github.com/gnea/grbl/wiki/Grbl-v1.1-Commands#x---kill-alarm-lock
        """
        ret = self.cmd("$X")
        assert(ret[-1]=='ok')
        
    def home(self):
        """ https://github.com/gnea/grbl/wiki/Grbl-v1.1-Commands#h---run-homing-cycle
        """
        self.write("$H")
        assert(ret[-1]=='ok')
        
# https://github.com/gnea/grbl/wiki/Grbl-v1.1-Configuration#---view-grbl-settings
settings = [
    ("$0", "step_pulse"),
    ("$1", "step_idle_delay"),
    ("$2", "step_port_invert"),
    ("$3", "direction_port_invert"),
    ("$4", "step_enable_invert"),
    ("$5", "limit_pin_invert"),
    ("$6", "probe_pin_invert"),
    ("$10", "status_report"),
    ("$11", "junction_deviation"),
    ("$12", "arc_tolerance"),
    ("$13", "report_inches"),
    ("$20", "soft_limits"),
    ("$21", "hard_limits"),
    ("$22", "homing_cycle"),
    ("$23", "homing_dir_invert"),
    ("$24", "homing_feed"),
    ("$25", "homing_seek"),
    ("$26", "homing_debounce"),
    ("$27", "homing_pull_off"),
    ("$30", "max_spindle_speed"),
    ("$31", "min_spindle_speed"),
    ("$32", "laser_mode"),
    ("$100", "x_steps_mm"),
    ("$101", "y_steps_mm"),
    ("$102", "z_steps_mm"),
    ("$110", "x_max_rate"),
    ("$111", "y_max_rate"),
    ("$112", "z_max_rate"),
    ("$120", "x_acceleration"),
    ("$121", "y_acceleration"),
    ("$122", "z_acceleration"),
    ("$130", "x_travel"),
    ("$131", "y_travel"),
    ("$132", "z_travel"),
    ]

def grbl_getter_generator(cmd):
    def grbl_getter(self):
        config = self.cmd("$$", resp=True, multiline=True)
        for config_line in config:
            if config_line.startswith("$"):
                key, value = config_line.split("=")
                if key == cmd:
                    return float(value)
        return None
    return grbl_getter
    
def grbl_setter_generator(cmd):
    def grbl_setter(self, value):
        set_cmd = "{cmd}={value}".format(cmd=cmd, value=value)
        ret = self.cmd(set_cmd, resp=True, multiline=False)
        print(ret)
        
    return grbl_setter

for setting in settings:
    cmd = setting[0]
    name = setting[1]
    
    setter = grbl_setter_generator(cmd)
    getter = grbl_getter_generator(cmd)
    
    prop = property(fget=getter,
                    fset=setter,
                    doc=" ".join(name.split("_")))
    
    setattr(GRBL, name, prop)
    
# https://github.com/gnea/grbl/wiki/Grbl-v1.1-Commands#---view-gcode-parameters
gcode_parameters = [
    "G54",
    "G55",
    "G56",
    "G57",
    "G58",
    "G59",
    "G28",
    "G30",
    "G92",
    "TLO",
    "PRB",
    ]


def gcode_param_gen(parameter):
    def gcode_param(self):
        gcode_parameters = self.cmd("$#") # View gcode parameters
        for gcode_parameter in gcode_parameters: 
            if parameter in gcode_parameter:
                _, value = gcode_parameter.split(":")
                value = value.strip("]")
                values = value.split(",")
                values = [float(value) for value in values]
                
                return values
        return None
    return gcode_param


for parameter in gcode_parameters:
    fcn = gcode_param_gen(parameter)
    prop = property(fget=fcn)
    setattr(GRBL, parameter, prop)

Overwriting GRBL.py


In [48]:
%aimport GRBL

In [68]:
grbl.serial.close()

In [69]:
grbl = GRBL.GRBL("/dev/cnc_3018")

In [71]:
grbl.reset()

In [72]:
# Metric
grbl.cmd("G21")
# Relative
grbl.cmd("G91")

['ok', 'ok']

# "Laser"

In [73]:
grbl.laser_mode

1.0

In [74]:
grbl.laser_mode=1

ok


In [75]:
g = grbl.cmd

In [76]:
g("G1 F500")

['ok', 'ok']

In [77]:
g("G1 X0")

['ok', 'ok']

In [78]:
g("G0 X10")

['ok', 'ok']

In [87]:
g("M3 S01")

['ok', 'ok']

In [86]:
g("M5")
g("M4 S100")
g("G1 X25")
g("G1 Y25")
g("G1 X-25")
g("G1 Y-25")
g("M5")

['ok']

In [27]:
g("M5")
g("M4 S50")
g("M5")
g("M4 S10")
g("G1 X100")
g("G1 Y100")

g("G1 X-100")
g("G1 Y-100")
g("M5")

['ok', 'ok']

In [61]:
g("M4 S100")
g("G1 X100 F500")
g("G1 Y100 F500")
g("G1 X-100 F500")
g("G1 Y-100 F500")
g("M5")

['ok', 'error:9']

In [88]:
g("G0 X-200")

['ok', 'ok']

In [90]:
g("G0 Y25")

['ok', 'ok']

In [91]:
g("M4 S10")
g("G1 X100 F500")
g("G1 Y100 F500")
g("G1 X-100 F500")
g("G1 Y-100 F500")
g("M5")

['ok']

In [95]:
def square(sides=25, power=10):
    g("M5")
    g("M4 S{:03d}".format(power))
    g("G1 X0")
    g("G4 P10")
    g("G1 X{}".format(sides))
    g("G1 Y{}".format(sides))
    g("G1 X-{}".format(sides))
    g("G1 Y-{}".format(sides))
    g("M5")

In [93]:
square(power=10)

In [96]:
square(power=50)

In [97]:
grbl.serial.close()